<a href="https://colab.research.google.com/github/WittJonas/bundestag/blob/main/bundestag_getdata_and_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re   
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.style.use('fivethirtyeight')

In [ ]:
consumerKey = '***'
consumerSecret = '***'
accessToken = '***'
accessTokenSecret = '***'

In [ ]:
# authenticate to Twitter
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [ ]:
pull_linke = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name ="Linksfraktion", tweet_mode = "extended").items(20000):
  pull_linke.append(tweet)

pull_gruene = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name ="GrueneBundestag", tweet_mode = "extended").items(20000):
  pull_gruene.append(tweet)

pull_spd = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name ="spdbt", tweet_mode = "extended").items(20000):
  pull_spd.append(tweet)

pull_cdu = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name ="cducsubt", tweet_mode = "extended").items(20000):
  pull_cdu.append(tweet)

pull_fdp = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name ="fdpbt", tweet_mode = "extended").items(20000):
  pull_fdp.append(tweet)

pull_afd = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name ="AfDimBundestag", tweet_mode = "extended").items(20000):
  pull_afd.append(tweet)

In [ ]:
for tweet in pull_linke[0:10]:
  print(tweet.full_text + '\n')

In [ ]:
# create dataframes
linke = pd.DataFrame([tweet.full_text for tweet in pull_linke], columns=['Tweets'])
gruene = pd.DataFrame([tweet.full_text for tweet in pull_gruene], columns=['Tweets'])
spd = pd.DataFrame([tweet.full_text for tweet in pull_spd], columns=['Tweets'])
cdu = pd.DataFrame([tweet.full_text for tweet in pull_cdu], columns=['Tweets'])
fdp = pd.DataFrame([tweet.full_text for tweet in pull_fdp], columns=['Tweets'])
afd = pd.DataFrame([tweet.full_text for tweet in pull_afd], columns=['Tweets'])

In [ ]:
# exclude retweets and reindex dataframe

parties=[linke, gruene, spd, cdu, fdp, afd]

for df in parties:
  df = df[~df.Tweets.str.startswith('RT')] # exclude retweets
  df = df.reset_index(drop=True) # reindex dataframe
  print(df.info())

In [ ]:
cdu.Tweets

0       Heute vor 60 Jahren begann der Mauerbau. Für e...
1       🔵Presseinfo👉_connemanEinsatz für SED-Opfer dar...
2       Eine wichtige Errungenschaft der vergangenen 4...
3       2019 haben wir als Fraktion mit dem LeipzigerA...
4       Morgen jährt sich der Beginn des Mauerbau|s in...
                              ...                        
1664    Geschickt in Infrastrukturprojekte investieren...
1665    🔵 Pressemitteilung 👉 Heute ist der Internation...
1666    Alfred Dregger war eine der prägenden Persönli...
1667    🔵Pressemitteilung ▶ Die Koalitionsfraktionen h...
1668    12 Milliarden Euro für Familien, Senioren oder...
Name: Tweets, Length: 1669, dtype: object

In [ ]:
# # define function to clean the text
# def cleanTxt(text):
#   text = re.sub('@[A-Za-z0-9]+', '', text) #remove @mentions
#   text = re.sub(r'#', '', text) #remove '#' symbol
#   text = re.sub(r'„', '', text) #remove '„' symbol
#   text = re.sub(r'“', '', text) #remove '“' symbol
#   # text = re.sub(r'+++', '', text) #remove '+++' symbol
#   text = re.sub(r'&amp', '', text) #remove '&' symbol
#   text = re.sub(r'.: ', '', text) #remove '.: '
#   text = re.sub(r'🔵 ', '', text) #remove '🔵'
#   text = re.sub(r'👉', '', text) #remove '👉'
#   text = re.sub(r'https?:\/\/\S+', '', text) #remove hyper link
#   text = re.sub(r'\n', ' ', text) #remove '/n'
  
#   return text

In [ ]:
# define function to clean the text

def cleanTxt(text):

  # remove all kinds of emojies 
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  emoji_pattern.sub(r'', text)

  # list of symbols / expressions / words that occur frequently in tweets that should be deleted
  removal_list = ['Pressemitteilung', 'PRESSESTATEMENT', 'PRESSEMITTEILUNG', 'Presseinfo', '@[A-Za-z0-9]+', '@_[A-Za-z0-9]+', 
                  '#', '„', '“', '&amp', '.: ', 'https?:\/\/\S+', '\n', '_', ' , , ,']
  for i in removal_list:
    text = text = re.sub(i, '', text)
  return text

In [ ]:
# apply function to parties

parties=[linke, gruene, spd, cdu, fdp, afd]

for df in parties:
  df.Tweets = df.Tweets.apply(cleanTxt)
  print(df.info())

In [ ]:
# replace empty strings with NA and drop NA
linke = linke.replace(r'^\s*$', np.nan, regex=True)
linke = linke.dropna()
gruene = gruene.replace(r'^\s*$', np.nan, regex=True)
gruene = gruene.dropna()
spd = spd.replace(r'^\s*$', np.nan, regex=True)
spd = spd.dropna()
cdu = cdu.replace(r'^\s*$', np.nan, regex=True)
cdu = cdu.dropna()
fdp = fdp.replace(r'^\s*$', np.nan, regex=True)
fdp = fdp.dropna()
afd = afd.replace(r'^\s*$', np.nan, regex=True)
afd = afd.dropna()

In [ ]:
# replace empty strings with tweet length < 20 char
for df in parties:
  index_names = df[df.Tweets.str.len() < 20].index # get indices of short tweets
  df.drop(index_names, inplace = True) # drop rows
  # print(df.info())

# Reindex dataframes
linke = linke.reset_index(drop=True)
gruene = gruene.reset_index(drop=True)
spd = spd.reset_index(drop=True)
cdu = cdu.reset_index(drop=True)
fdp = fdp.reset_index(drop=True)
afd = afd.reset_index(drop=True)

In [ ]:
# Combine to one dataframe and generate label column for later training
linke['label'] = 0 
gruene['label'] = 1
spd['label'] = 2 
cdu['label'] = 3
fdp['label'] = 4 
afd['label'] = 5

frames = [linke, gruene, spd, cdu, fdp, afd]
data = pd.concat(frames)

train, test = train_test_split(data, test_size=0.2)

# Reindex dataframes
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
# mount drive to save data
from google.colab import drive
drive.mount('drive')

In [ ]:
# save data individually
linke.to_csv('linke_origin_b.csv', index=False)
!cp linke_origin_b.csv "drive/My Drive/NLP_project"

gruene.to_csv('gruene_origin_b.csv', index=False)
!cp gruene_origin_b.csv "drive/My Drive/NLP_project"

spd.to_csv('spd_origin_b.csv', index=False)
!cp spd_origin_b.csv "drive/My Drive/NLP_project"

cdu.to_csv('cdu_origin_b.csv', index=False)
!cp cdu_origin_b.csv "drive/My Drive/NLP_project"

fdp.to_csv('fdp_origin_b.csv', index=False)
!cp fdp_origin_b.csv "drive/My Drive/NLP_project"

afd.to_csv('afd_origin_b.csv', index=False)
!cp afd_origin_b.csv "drive/My Drive/NLP_project"

#save data concatenated
test.to_csv('test.csv', index=False)
!cp test.csv "drive/My Drive/NLP_project"

train.to_csv('train.csv', index=False)
!cp train.csv "drive/My Drive/NLP_project"

In [ ]:
# load data
linke = pd.read_csv("drive/My Drive/NLP_project/linke_origin_b.csv")
gruene = pd.read_csv("drive/My Drive/NLP_project/gruene_origin_b.csv")
spd = pd.read_csv("drive/My Drive/NLP_project/spd_origin_b.csv")
cdu = pd.read_csv("drive/My Drive/NLP_project/cdu_origin_b.csv")
fdp = pd.read_csv("drive/My Drive/NLP_project/fdp_origin_b.csv")
afd = pd.read_csv("drive/My Drive/NLP_project/afd_origin_b.csv")